#   Q18 Machine Learning Rolling Basis

In this example we predict whether the price will rise or fall by using supervised learning (Bayesian Ridge Regression). This template represents a starting point for developing a system which can take part to the **Q18 NASDAQ-100 Stock Long-Short contest**.

It consists of two parts.

* In the **first part** we just perform a global training of the time series using all time series data. We disregard the sequential aspect of the data and use also future data to train past data.

* In the **second part** we use the built-in backtester and perform training and prediction on a rolling basis in order to avoid forward looking. Please note that we are using a **specialized** version of the Quantiacs backtester which dramatically speeds up the the backtesting process by retraining your model on a regular basis.

**Features for learning**: we will use several technical indicators trying to capture different features. You can have a look at [**Technical Indicators**](https://quantiacs.com/documentation/en/user_guide/technical_indicators.html).

Please note that:

* Your trading algorithm can open short and long positions.

* At each point in time your algorithm can trade all or a subset of the stocks which at that point of time are or were part of the NASDAQ-100 stock index. Note that the composition of this set changes in time, and Quantiacs provides you with an appropriate filter function for selecting them.

* The Sharpe ratio of your system since January 1st, 2006, has to be larger than 1.

* Your system cannot be a copy of the current examples. We run a correlation filter on the submissions and detect duplicates.

* For simplicity we will use a single asset. It pays off to use more assets, ideally uncorrelated, and diversify your positions for a more solid Sharpe ratio.

More details on the rules can be found [here](https://quantiacs.com/contest).

**Need help?** Check the [**Documentation**](https://quantiacs.com/documentation/en/) and find solutions/report problems in the [**Forum**](https://quantiacs.com/community/categories) section.

**More help with Jupyter?** Check the official [**Jupyter**](https://jupyter.org/) page.

Once you are done, click on **Submit to the contest** and take part to our competitions.

API reference:

* **data**: check how to work with [data](https://quantiacs.com/documentation/en/reference/data_load_functions.html);

* **backtesting**: read how to run the [simulation](https://quantiacs.com/documentation/en/reference/evaluation.html) and check the results.

Need to use the optimizer function to automate tedious tasks?

* **optimization**: read more on our [article](https://quantiacs.com/community/topic/29/optimizing-and-monitoring-a-trading-system-with-quantiacs).

#   Q18 Machine Learning Rolling Basis MOD

## Author: Manuel Quintana

This notebook is a modification of the **Q18 NASDAQ-100 Stock Long-Short contest**. Where instead of analyzing the **Nasdaq 100** market, the **SP&500** is chosen, taking into account historical data since 2005.
For this purpose, 20 assets are selected and their selection will be explained later, together with the selection of the model.

En la primera parte 

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import logging

import xarray as xr  # xarray for data manipulation

import qnt.data as qndata     # functions for loading data
import qnt.backtester as qnbt # built-in backtester
import qnt.ta as qnta         # technical analysis library
import qnt.stats as qnstats   # statistical functions

import pandas as pd           # data manipulation
import numpy as np            # numerical functions

import matplotlib.pyplot as plt  # graphing

np.seterr(divide = "ignore")

from qnt.ta.macd import macd # moving average convergence divergence
from qnt.ta.rsi  import rsi  # relative strength index
from qnt.ta.stochastic import stochastic_k, stochastic, slow_stochastic # stochastic oscillator


from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor

NOTICE: The environment variable DATA_BASE_URL was not specified. The default value is 'https://data-api.quantiacs.io/'
NOTICE: The environment variable CACHE_RETENTION was not specified. The default value is '7'
NOTICE: The environment variable CACHE_DIR was not specified. The default value is 'data-cache'


The selection of these 20 assets for the strategy is based on a combination of fundamental and technical criteria aimed at maximizing diversification, liquidity, and market relevance. Here is the rationale behind this selection:

### 1. **Sector Diversification**
   - The assets belong to different key economic sectors, reducing dependency on a single sector and mitigating specific risks:
     - **Technology**: AAPL (Apple), MSFT (Microsoft), GOOGL (Alphabet), AMZN (Amazon), FB (Meta Platforms).
     - **Healthcare**: JNJ (Johnson & Johnson), PFE (Pfizer), ABBV (AbbVie), MRK (Merck & Co.).
     - **Financials**: JPM (JPMorgan Chase), BRK.B (Berkshire Hathaway).
     - **Consumer**: PG (Procter & Gamble), KO (Coca-Cola), PEP (PepsiCo), HD (Home Depot), DIS (Disney).
     - **Energy**: XOM (Exxon Mobil).
     - **Services and Others**: V (Visa), MA (Mastercard), UNH (UnitedHealth Group).

### 2. **High Market Capitalization**
   - All the selected assets are **blue-chip stocks**, meaning well-established companies with high market capitalization. This ensures:
     - High **liquidity**, allowing easy entry and exit of positions without significantly impacting the price.
     - Greater price stability compared to smaller assets.

### 3. **Relevance in the S&P 500**
   - These assets represent some of the most influential companies in the S&P 500 index, with significant weight in its calculation.

### 4. **Historical Growth and Profitability**
   - Assets like AAPL, MSFT, and AMZN have consistently proven to be market growth drivers.
   - Companies like JNJ and KO are known for their stability and regular dividends, providing a balance between growth and defensiveness.

### 5. **Controlled Volatility**
   - The combination includes both growth companies (e.g., AMZN, GOOGL) and value companies (e.g., PG, KO), balancing volatility and returns.

### 6. **Adaptability to Quantitative Strategies**
   - These assets are widely used in quantitative strategies due to their extensive data history, high trading frequency, and predictable patterns.

In summary, this combination of assets maximizes sector diversification and ensures liquidity and stability while balancing the risk between growth and value companies. It also provides a set of assets that are highly representative of the market.

In [4]:
# # load SP500 data
# stock_data = qndata.stocks.load_spx_data(min_date="2005-06-01")
# # assets list
# assets = ["SPY:AAPL", "SPY:MSFT", "SPY:GOOGL", "SPY:AMZN", "SPY:FB", "SPY:BRK.B", "SPY:JNJ", "SPY:V", "SPY:PG", "SPY:JPM", "SPY:UNH", "SPY:HD", "SPY:MA", "SPY:PFE", "SPY:ABBV", "SPY:MRK", "SPY:PEP", "SPY:KO", "SPY:DIS", "SPY:XOM"]

# ma_50 = stock_data.sel(field="close").rolling(time=50).mean().expand_dims('field').assign_coords(field=['ma_50'])
# ma_200 = stock_data.sel(field="close").rolling(time=200).mean().expand_dims('field').assign_coords(field=['ma_200'])
# volatility = stock_data.sel(field="close").rolling(time=50).std().expand_dims('field').assign_coords(field=['volatility'])

# stock_data = xr.concat([stock_data, ma_50, ma_200, volatility], dim='field')

# Cargar datos del S&P 500 desde Quantiacs
# Cargar datos del S&P 500 desde Quantiacs
stock_data = qndata.stocks.load_spx_data(min_date="2005-06-01")

# Filtrar activos líquidos por volumen promedio
liquidity_threshold = 1_000_000  # Umbral de liquidez
avg_volume = stock_data.sel(field="vol").mean("time")
liquid_assets = avg_volume.where(avg_volume > liquidity_threshold, drop=True).asset.values

# Seleccionar solo activos líquidos
stock_data = stock_data.sel(asset=liquid_assets)

print(f"Activos líquidos seleccionados: {list(stock_data.asset.values)}")

# Calcular medias móviles e indicadores adicionales
ma_50 = stock_data.sel(field="close").rolling(time=50).mean().expand_dims('field').assign_coords(field=['ma_50'])
ma_200 = stock_data.sel(field="close").rolling(time=200).mean().expand_dims('field').assign_coords(field=['ma_200'])

# Calcular ATR (Average True Range) solo una vez
atr = qnta.atr(stock_data.sel(field="high"), stock_data.sel(field="low"), stock_data.sel(field="close"), 14)
atr = atr.expand_dims('field').assign_coords(field=['atr'])

# Cálculo manual de Bandas de Bollinger
# def compute_bollinger_bands(prices, window=20):
#     moving_avg = prices.rolling(time=window).mean()
#     moving_std = prices.rolling(time=window).std()
    
#     bollinger_up = moving_avg + (moving_std * 2)
#     bollinger_low = moving_avg - (moving_std * 2)
    
#     return bollinger_up, moving_avg, bollinger_low

# bollinger_up, bollinger_mid, bollinger_low = compute_bollinger_bands(stock_data.sel(field="close"))

# bollinger_up = bollinger_up.expand_dims('field').assign_coords(field=['bollinger_up'])
# bollinger_low = bollinger_low.expand_dims('field').assign_coords(field=['bollinger_low'])
# bollinger_mid = bollinger_mid.expand_dims('field').assign_coords(field=['bollinger_mid'])

volatility = stock_data.sel(field="close").rolling(time=50).std().expand_dims('field').assign_coords(field=['volatility'])

# Concatenar todas las características al dataset original
stock_data = xr.concat([stock_data, atr, ma_50, ma_200, volatility], dim='field')

print("Campos disponibles después de la concatenación:", stock_data.field.values)


print("Variables añadidas:", stock_data.field.values)

# Concatenar todas las características al dataset original
# stock_data = xr.concat([stock_data, ma_50, ma_200, volatility, atr, bollinger_up, bollinger_low], dim='field')



fetched chunk 1/13 0s
fetched chunk 2/13 0s
fetched chunk 3/13 0s
fetched chunk 4/13 0s
fetched chunk 5/13 0s
fetched chunk 6/13 0s
fetched chunk 7/13 0s
fetched chunk 8/13 0s
fetched chunk 9/13 0s
fetched chunk 10/13 1s
fetched chunk 11/13 1s
fetched chunk 12/13 1s
fetched chunk 13/13 1s
Data loaded 1s
Activos líquidos seleccionados: ['NAS:AAL', 'NAS:AAPL', 'NAS:ABNB', 'NAS:ADBE', 'NAS:ADI', 'NAS:ADP', 'NAS:ADSK', 'NAS:AEP', 'NAS:AKAM', 'NAS:AMAT', 'NAS:AMD', 'NAS:AMGN', 'NAS:AMZN', 'NAS:APA', 'NAS:AVGO', 'NAS:AXON', 'NAS:BIIB', 'NAS:BKR', 'NAS:CDNS', 'NAS:CEG', 'NAS:CHRW', 'NAS:CHTR', 'NAS:CMCSA', 'NAS:CME', 'NAS:COO', 'NAS:COST', 'NAS:CPB', 'NAS:CPRT', 'NAS:CRWD', 'NAS:CSCO', 'NAS:CSGP', 'NAS:CSX', 'NAS:CTAS', 'NAS:CTSH', 'NAS:CZR', 'NAS:DELL', 'NAS:DLTR', 'NAS:DXCM', 'NAS:EA', 'NAS:EBAY', 'NAS:ENPH', 'NAS:ETSY', 'NAS:EVRG', 'NAS:EXC', 'NAS:EXPE', 'NAS:FANG', 'NAS:FAST', 'NAS:FFIV', 'NAS:FITB', 'NAS:FOX', 'NAS:FOXA', 'NAS:FSLR', 'NAS:FTNT', 'NAS:GEHC', 'NAS:GEN', 'NAS:GILD', 'NAS:GO

In [5]:
# def get_features(data):
#     """Builds the features used for learning:
#        * a trend indicator;
#        * the moving average convergence divergence;
#        * a volatility measure;
#        * the stochastic oscillator;
#        * the relative strength index;
#        * the logarithm of the closing price.
#        These features can be modified and new ones can be added easily.
#     """

#     # trend:
#     trend = qnta.roc(qnta.lwma(data.sel(field="close"), 60), 1)

#     # moving average convergence  divergence (MACD):
#     macd = qnta.macd(data.sel(field="close"))
#     macd2_line, macd2_signal, macd2_hist = qnta.macd(data, 12, 26, 9)

#     # volatility:
#     volatility = qnta.tr(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"))
#     volatility = volatility / data.sel(field="close")
#     volatility = qnta.lwma(volatility, 14)

#     # the stochastic oscillator:
#     k, d = qnta.stochastic(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14)

#     # the relative strength index:
#     rsi = qnta.rsi(data.sel(field="close"))

#     # the logarithm of the closing price:
#     price = data.sel(field="close").ffill("time").bfill("time").fillna(0) # fill NaN
#     price = np.log(price)

#     # combine the six features:
#     result = xr.concat(
#         [trend, macd2_signal.sel(field="close"), volatility,  d, rsi, price],
#         pd.Index(
#             ["trend",  "macd", "volatility", "stochastic_d", "rsi", "price"],
#             name = "field"
#         )
#     )

#     return result.transpose("time", "field", "asset")

# def get_features(data):
#     """Construye características utilizadas para el modelo:
#        * Indicador de tendencia;
#        * MACD;
#        * Volatilidad;
#        * Oscilador estocástico;
#        * RSI;
#        * Precio logarítmico;
#        * ATR;
#        * Bandas de Bollinger (superior e inferior).
#     """

#     # Indicadores calculados
#     trend = qnta.roc(qnta.lwma(data.sel(field="close"), 60), 1)
#     macd2_line, macd2_signal, macd2_hist = qnta.macd(data, 12, 26, 9)
#     volatility = qnta.lwma(data.sel(field="volatility"), 14)
#     k, d = qnta.stochastic(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14)
#     rsi = qnta.rsi(data.sel(field="close"))
#     price = np.log(data.sel(field="close").ffill("time").bfill("time").fillna(0))

#     # Incorporar nuevas variables
#     atr = data.sel(field="atr")
#     bollinger_up = data.sel(field="bollinger_up")
#     bollinger_low = data.sel(field="bollinger_low")

#     # Combinar todas las características
#     result = xr.concat(
#         [trend, macd2_signal.sel(field="close"), volatility, d, rsi, price, atr, bollinger_up, bollinger_low],
#         pd.Index(["trend", "macd", "volatility", "stochastic_d", "rsi", "price", "atr", "bollinger_up", "bollinger_low"],
#                  name="field")
#     )

#     return result.transpose("time", "field", "asset")


def get_features(data):
    """Construye características utilizadas para el modelo incluyendo ATR."""

    # required_fields = ["close", "high", "low", "bollinger_up", "bollinger_low"]
    # for field in required_fields:
    #     if field not in data.field.values:
    #         raise ValueError(f"El campo '{field}' no está presente en los datos.")

    # Asegurarnos de que el ATR está disponible
    if 'atr' not in data.field.values:
        atr = qnta.atr(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14)
        atr = atr.expand_dims('field').assign_coords(field=['atr'])
        data = xr.concat([data, atr], dim='field')

    # Indicadores calculados
    trend = qnta.roc(qnta.lwma(data.sel(field="close"), 60), 1)
    macd2_line, macd2_signal, macd2_hist = qnta.macd(data, 12, 26, 9)
    volatility = qnta.lwma(data.sel(field="atr"), 14)
    k, d = qnta.stochastic(data.sel(field="high"), data.sel(field="low"), data.sel(field="close"), 14)
    rsi = qnta.rsi(data.sel(field="close"))
    price = np.log(data.sel(field="close").ffill("time").bfill("time").fillna(0))

    # Incorporar nuevas variables
    atr = data.sel(field="atr")
    # bollinger_up = data.sel(field="bollinger_up")
    # bollinger_low = data.sel(field="bollinger_low")

    # Combinar todas las características
    result = xr.concat(
        [trend, macd2_signal.sel(field="close"), volatility, d, rsi, price, atr],
        pd.Index(["trend", "macd", "atr", "stochastic_d", "rsi", "price", "atr"],
                 name="field")
    )
    #     result = xr.concat(
#         [trend, macd2_signal.sel(field="close"), volatility, d, rsi, price, atr, bollinger_up, bollinger_low],
#         pd.Index(["trend", "macd", "volatility", "stochastic_d", "rsi", "price", "atr", "bollinger_up", "bollinger_low"],
#                  name="field")
#     )


    return result.transpose("time", "field", "asset")




In [6]:
print(stock_data.field.values)


['open' 'low' 'high' 'close' 'vol' 'divs' 'split_cumprod' 'is_liquid'
 'atr' 'ma_50' 'ma_200' 'volatility']


In [7]:
# # displaying the features:
# my_features = get_features(stock_data)
# display(my_features.sel(field="trend").to_pandas())

# Verificar los campos disponibles en los datos
print("Campos disponibles en stock_data:", stock_data.field.values)

# Imputar valores faltantes
stock_data = stock_data.ffill("time").bfill("time").fillna(0)

# Generar características
my_features = get_features(stock_data)

# Visualizar la tendencia del primer activo
asset_example = stock_data.asset.values[0]
trend_data = my_features.sel(field="trend", asset=asset_example)
display(trend_data.to_pandas().head(20))

# Descripción de los datos para verificar su validez
print(trend_data.to_pandas().describe())


Campos disponibles en stock_data: ['open' 'low' 'high' 'close' 'vol' 'divs' 'split_cumprod' 'is_liquid'
 'atr' 'ma_50' 'ma_200' 'volatility']


time
2005-06-01   NaN
2005-06-02   NaN
2005-06-03   NaN
2005-06-06   NaN
2005-06-07   NaN
2005-06-08   NaN
2005-06-09   NaN
2005-06-10   NaN
2005-06-13   NaN
2005-06-14   NaN
2005-06-15   NaN
2005-06-16   NaN
2005-06-17   NaN
2005-06-20   NaN
2005-06-21   NaN
2005-06-22   NaN
2005-06-23   NaN
2005-06-24   NaN
2005-06-27   NaN
2005-06-28   NaN
dtype: float64

count    4880.000000
mean       -0.007892
std         0.324747
min        -2.230107
25%        -0.063792
50%         0.000000
75%         0.045041
max         2.620239
dtype: float64


In [8]:
# def get_target_classes(data):
#     """ Target classes for predicting if price goes up or down."""

#     price_current = data.sel(field="close")
#     price_future  = qnta.shift(price_current, -1)

#     class_positive = 1 # prices goes up
#     class_neutral  = 0 # price stays the same
#     class_negative = -1 # price goes down

#     target_price_up = xr.where(price_future > price_current, class_positive, class_negative)

#     return target_price_up

# def get_target_classes(data):
#     """ Target classes for predicting if price goes up or down."""

#     daily_return = qnta.change(data.sel(field="close"))/(data.sel(field="close"))
#     daily_return_future = daily_return.shift(time=-1)

#     move = 0.005

#     class_positive = 1 # prices goes up
#     class_neutral  = 0 # price stays the same
#     class_negative = -1 # price goes down

#     target_price = xr.where(daily_return_future < -move, class_negative, daily_return_future)
#     target_price = xr.where(target_price > move, class_positive, target_price)
#     target_price = xr.where(abs(target_price) !=1, class_neutral, target_price)
    
#     return target_price

# def get_target_classes(data):
#     """ Target classes for predicting if price goes up or down based on new criteria."""

#     daily_return = qnta.change(data.sel(field="close")) / data.sel(field="close")
#     daily_return_future = daily_return.shift(time=-1)

#     move = 0.001  # 0.1% threshold

#     class_positive = 1  # Price goes up more than 0.1%
#     class_neutral  = 0  # Price changes within ±0.1%
#     class_negative = -1  # Price goes down more than 0.1%

#     # Applying the new classification rules
#     target_price = xr.where(daily_return_future < -move, class_negative, daily_return_future)
#     target_price = xr.where(target_price > move, class_positive, target_price)
#     target_price = xr.where(abs(target_price) != 1, class_neutral, target_price)

#     return target_price

def get_target_classes(data):
    """ Target classes for predicting if price goes up or down based on new criteria."""

    daily_return = qnta.change(data.sel(field="close")) / data.sel(field="close")
    daily_return_future = daily_return.shift(time=-1)

    move = 0.001  # 0.1% threshold

    class_positive = 1  # Price goes up more than 0.1%
    # class_neutral  = 0  # Price changes within ±0.1%
    class_negative = 0  # Price goes down more than 0.1%

    # Applying the new classification rules
    target_price = xr.where(daily_return_future < -move, class_negative, daily_return_future)
    target_price = xr.where(target_price > move, class_positive, target_price)
    # target_price = xr.where(abs(target_price) != 1, class_neutral, target_price)

    return target_price


In [9]:
# displaying the target classes:
my_targetclass = get_target_classes(stock_data)
display(my_targetclass.to_pandas())

asset,NAS:AAL,NAS:AAPL,NAS:ABNB,NAS:ADBE,NAS:ADI,NAS:ADP,NAS:ADSK,NAS:AEP,NAS:AKAM,NAS:AMAT,...,NYS:WM,NYS:WMB,NYS:WMT,NYS:WRB,NYS:WY,NYS:XOM,NYS:XYL,NYS:YUM,NYS:ZBH,NYS:ZTS
time,,,,,,,,,,,,,,,,,,,,,
2005-06-01,0.0,0.0,0.0,0.0,1.000000,1.0,0.000000,0.000000,0.000000,1.000000,...,0.0,0.000000,1.000000,0.0,-0.000753,1.0,0.0,1.000000,1.0,0.0
2005-06-02,0.0,0.0,0.0,0.0,-0.000792,0.0,0.000000,-0.000559,0.000000,0.000593,...,1.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0
2005-06-03,0.0,0.0,0.0,0.0,0.000000,1.0,1.000000,1.000000,1.000000,0.000000,...,0.0,1.000000,1.000000,1.0,1.000000,1.0,0.0,-0.000535,0.0,0.0
2005-06-06,0.0,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000,1.000000,0.000000,...,0.0,0.000000,0.000628,1.0,1.000000,0.0,0.0,1.000000,1.0,0.0
2005-06-07,0.0,1.0,0.0,0.0,1.000000,0.0,0.000000,0.000558,0.000000,1.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-10,0.0,0.0,0.0,1.0,1.000000,1.0,-0.000492,1.000000,1.000000,0.000000,...,1.0,1.000000,0.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0
2025-01-13,1.0,0.0,0.0,1.0,1.000000,1.0,1.000000,-0.000423,1.000000,1.000000,...,0.0,1.000000,0.000000,1.0,1.000000,1.0,0.0,1.000000,0.0,0.0
2025-01-14,0.0,1.0,1.0,1.0,1.000000,1.0,1.000000,1.000000,-0.000663,1.000000,...,1.0,-0.000862,1.000000,1.0,1.000000,1.0,1.0,1.000000,1.0,1.0


### **Model Selection: Gradient Boosting Regressor**

The **Gradient Boosting Regressor** was chosen for this project because of its ability to handle complex relationships in the data and provide highly accurate predictions. Here are the reasons for this choice:

1. **Ability to Handle Non-Linear Data**:
   - Financial data often exhibit non-linear relationships between input features and target variables. Gradient Boosting is designed to capture these non-linear patterns effectively.

2. **Robustness**:
   - Gradient Boosting builds the predictive model iteratively, reducing errors from previous iterations. This makes it robust to overfitting, especially with well-tuned hyperparameters.

3. **Customizable Parameters**:
   - The algorithm offers flexibility through hyperparameters like the number of boosting stages (`n_estimators`), learning rate, and maximum depth of trees (`max_depth`), which were fine-tuned here for better performance on financial data.

4. **High Accuracy in Complex Datasets**:
   - Gradient Boosting is one of the most widely used algorithms for structured data due to its high accuracy in capturing intricate patterns.

5. **Interpretability**:
   - Gradient Boosting provides feature importance scores, making it easier to interpret which variables are driving the predictions.

---

### **Code Explanation**

#### **Model Definition**
The `get_model` function defines the **Gradient Boosting Regressor** with the following hyperparameters:
- **`n_estimators=200`**: Specifies the number of boosting stages (trees). A higher value can improve performance but increases training time.
- **`learning_rate=0.05`**: Shrinks the contribution of each tree, making the model learn more conservatively to avoid overfitting.
- **`max_depth=5`**: Limits the depth of the trees to control overfitting and reduce model complexity.
- **`min_samples_split=10`**: Sets the minimum number of samples required to split an internal node, ensuring splits happen only with sufficient data.
- **`min_samples_leaf=4`**: Specifies the minimum number of samples required at a leaf node, reducing variance in the predictions.
- **`random_state=42`**: Ensures reproducibility of results by fixing the randomness during model training.

#### **Model Training for Each Asset**
The subsequent code iteratively trains a model for each asset using the `Gradient Boosting Regressor`.

1. **Retrieve Data for Each Asset**:
   - For each asset in the dataset (`asset_name_all`), the code extracts the corresponding **features** and **target** values, ensuring no missing values with `.dropna()`.

2. **Align Features and Targets**:
   - Ensures that features and target values are synchronized in terms of their time indices using `xr.align()`.

3. **Filter Insufficient Data**:
   - If the data for a given asset has fewer than 10 data points, the training process for that asset is skipped.

4. **Train the Model**:
   - The `Gradient Boosting Regressor` is instantiated via the `get_model()` function and trained using the feature and target data.

5. **Handle Exceptions**:
   - If model training fails for any reason (e.g., invalid data), the exception is logged, and the process continues with the next asset.

6. **Store the Model**:
   - Successfully trained models are stored in the dictionary `models`, with the asset name as the key and the trained model as the value.

#### **Output**
The `models` dictionary contains a separate trained model for each asset, enabling asset-specific predictions.


In [10]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

# def get_model():
#     """This is a constructor for the ML model (Random Forest Regressor) which can be easily
#        modified for using different models.
#     """

#     # Random Forest Regressor with tuned hyperparameters
#     model = RandomForestRegressor(
#         n_estimators=100,  # Number of trees in the forest
#         max_depth=10,  # Maximum depth of the tree
#         min_samples_split=10,  # Minimum samples required to split an internal node
#         min_samples_leaf=4,  # Minimum samples required to be at a leaf node
#         random_state=42  # Ensures reproducibility
#     )
#     return model

class RandomForestWithClassLabels:
    def __init__(self):
        self.model = RandomForestRegressor(
            n_estimators=200,
            max_depth=8,
            min_samples_split=5,
            min_samples_leaf=2,
            random_state=42,
            criterion='squared_error',
            n_jobs=-1
        )

    def fit(self, X, y):
        self.model.fit(X, y)

    def predict(self, X):
        # Predicción continua
        return self.model.predict(X)

    def predict_classes(self, X):
        # Asigna -1, 0 o 1 según criterio
        raw_preds = self.model.predict(X)
        return np.where(raw_preds > 0.0, 1,
                        np.where(raw_preds < 0.0, -1, 0))


def get_model():
    """Constructor del modelo ML (Random Forest Regressor) con una capa adicional
       para convertir las predicciones en valores -1, 0 o 1, en sintonía con
       get_target_classes.
    """
    model = RandomForestWithClassLabels()

    return model


# def get_model():
#     """This is a constructor for the ML model (Gradient Boosting Regressor) which can be easily
#        modified for using different models.
#     """

#     # Gradient Boosting Regressor with tuned hyperparameters
#     model = GradientBoostingRegressor(
#         n_estimators=200,  # Number of boosting stages to be run
#         learning_rate=0.05,  # Step size shrinkage
#         max_depth=5,  # Maximum depth of the individual regression estimators
#         min_samples_split=10,  # Minimum samples required to split an internal node
#         min_samples_leaf=4,  # Minimum samples required to be at a leaf node
#         random_state=42  # Ensures reproducibility
#     )
#     return model
# def get_model():
#     """This is a constructor for the ML model (Gradient Boosting Regressor) which can be easily
#        modified for using different models.
#     """

#     # Gradient Boosting Regressor with tuned hyperparameters
#     model = GradientBoostingRegressor(
#         n_estimators=500,  # Aumentar para mejorar precisión (más etapas de boosting)
#         learning_rate=0.001,  # Reducir para evitar overfitting y suavizar predicciones
#         max_depth=7,  # Aumentar profundidad para capturar más patrones
#         min_samples_split=5,  # Permitir más divisiones
#         min_samples_leaf=2,  # Reducir para hacer hojas más sensibles a cambios
#         subsample=0.8,  # Aplicar muestreo para reducir overfitting
#         random_state=42
#     )

#     return model


In [11]:
# # Create and train the models working on an asset-by-asset basis.

asset_name_all = stock_data.coords["asset"].values

models = dict()

for asset_name in asset_name_all:

        # drop missing values:
        target_cur   = my_targetclass.sel(asset=asset_name).dropna("time", "any")
        features_cur = my_features.sel(asset=asset_name).dropna("time", "any")

        # align features and targets:
        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        if len(features_cur.time) < 10:
            # not enough points for training
                continue

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model

        except:
            logging.exception("model training failed")

print(models)


# from sklearn.preprocessing import StandardScaler

# asset_name_all = stock_data.coords["asset"].values
# models = dict()

# for asset_name in asset_name_all:
#     try:
#         # Eliminar valores faltantes solo en la variable objetivo
#         target_cur = my_targetclass.sel(asset=asset_name).dropna("time", "any")
        
#         # Imputación de valores faltantes en las características
#         features_cur = my_features.sel(asset=asset_name).fillna(method="ffill").fillna(method="bfill")

#         # Alinear características y objetivo en tiempo
#         target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

#         if len(feature_for_learn_df.time) < 30:
#             logging.warning(f"Activo {asset_name} no tiene suficientes puntos para entrenamiento")
#             continue

#         # Normalización de características
#         scaler = StandardScaler()
#         X_scaled = scaler.fit_transform(feature_for_learn_df.to_pandas())

#         # Obtener el modelo y entrenarlo
#         model = get_model()
#         model.fit(X_scaled, target_for_learn_df.to_pandas().values.ravel())
        
#         # Guardar modelo entrenado
#         models[asset_name] = (model, scaler)

#     except Exception as e:
#         logging.exception(f"Fallo en entrenamiento del modelo para {asset_name}: {e}")

# print(f"Modelos entrenados para {len(models)} activos")


{'NAS:AAL': <__main__.RandomForestWithClassLabels object at 0x7f997254c7c0>, 'NAS:AAPL': <__main__.RandomForestWithClassLabels object at 0x7f98eccfd540>, 'NAS:ABNB': <__main__.RandomForestWithClassLabels object at 0x7f98eccb1b70>, 'NAS:ADBE': <__main__.RandomForestWithClassLabels object at 0x7f98ec20ba90>, 'NAS:ADI': <__main__.RandomForestWithClassLabels object at 0x7f98ec20b100>, 'NAS:ADP': <__main__.RandomForestWithClassLabels object at 0x7f98ec20a8f0>, 'NAS:ADSK': <__main__.RandomForestWithClassLabels object at 0x7f98ec20b0a0>, 'NAS:AEP': <__main__.RandomForestWithClassLabels object at 0x7f98ec1c49d0>, 'NAS:AKAM': <__main__.RandomForestWithClassLabels object at 0x7f98ec27b1f0>, 'NAS:AMAT': <__main__.RandomForestWithClassLabels object at 0x7f98ec28e170>, 'NAS:AMD': <__main__.RandomForestWithClassLabels object at 0x7f98ec228ca0>, 'NAS:AMGN': <__main__.RandomForestWithClassLabels object at 0x7f98ec2784f0>, 'NAS:AMZN': <__main__.RandomForestWithClassLabels object at 0x7f98ec28f010>, 'NA

In [12]:
# Performs prediction and generates output weights:

asset_name_all = stock_data.coords["asset"].values
weights = xr.zeros_like(stock_data.sel(field="close"))

for asset_name in asset_name_all:
    if asset_name in models:
        model = models[asset_name]
        features_all = my_features
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")
        if len(features_cur.time) < 1:
            continue
        try:
            weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)
        except KeyboardInterrupt as e:
            raise e
        except:
            logging.exception("model prediction failed")

is_liquid = stock_data.sel(field="is_liquid")
weights = weights*is_liquid

print(weights)

<xarray.DataArray 'stocks_s&p500' (time: 4940, asset: 422)> Size: 17MB
array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.51079369, 0.56966724, ..., 0.69947513, 0.44855807,
        0.57208988],
       [0.        , 0.37802373, 0.40678802, ..., 0.65114058, 0.47153513,
        0.54062586],
       [0.        , 0.55006491, 0.43347533, ..., 0.63724846, 0.47253776,
        0.40615176]])
Coordinates:
  * time     (time) datetime64[ns] 40kB 2005-06-01 2005-06-02 ... 2025-01-16
  * asset    (asset) <U9 15kB 'NAS:AAL' 'NAS:AAPL' ... 'NYS:ZBH' 'NYS:ZTS'


In [13]:
def get_sharpe(stock_data, weights):
    """Calculates the Sharpe ratio"""
    rr = qnstats.calc_relative_return(stock_data, weights)
    sharpe = qnstats.calc_sharpe_ratio_annualized(rr).values[-1]
    return sharpe

sharpe = get_sharpe(stock_data, weights)
sharpe

1.494636808626906

The sharpe ratio using the method above follows from **forward looking**. Predictions for (let us say) 2017 know about the relation between features and targets in 2020. Let us visualize the results:

In [14]:
import qnt.graph as qngraph

statistics = qnstats.calc_stat(stock_data, weights)

display(statistics.to_pandas().tail())

performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())

# check for correlations with existing strategies:
qnstats.print_correlation(weights,stock_data)

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2025-01-10,197.635309,-0.014270,0.208791,-0.046806,-0.41692,1.483881,0.309821,1.0,421.0,0.091065,22.306237
2025-01-13,199.320482,0.008527,0.208776,-0.038678,-0.41692,1.486361,0.310317,1.0,421.0,0.091080,22.304485
2025-01-14,201.066177,0.008758,0.208762,-0.030259,-0.41692,1.488911,0.310828,1.0,421.0,0.091085,22.302360
2025-01-15,203.257745,0.010900,0.208753,-0.019689,-0.41692,1.492109,0.311482,1.0,421.0,0.091090,22.300057
2025-01-16,205.022354,0.008682,0.208738,-0.011178,-0.41692,1.494637,0.311988,1.0,421.0,0.091096,22.278854


time,2025-01-16
field,
sharpe_ratio,1.494637


NOTICE: The environment variable ENGINE_CORRELATION_URL was not specified. The default value is 'https://quantiacs.io/referee/submission/forCorrelation'
NOTICE: The environment variable STATAN_CORRELATION_URL was not specified. The default value is 'https://quantiacs.io/statan/correlation'
NOTICE: The environment variable PARTICIPANT_ID was not specified. The default value is '0'



Ok. This strategy does not correlate with other strategies.


In [15]:
"""R2 (coefficient of determination) regression score function."""
r2_score(my_targetclass, weights, multioutput="variance_weighted")

ValueError: Input contains NaN.

In [ ]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
explained_variance_score(my_targetclass, weights, multioutput="uniform_average")


0.14520337279843384

In [17]:
"""The explained variance score explains the dispersion of errors of a given dataset"""
mean_absolute_error(my_targetclass, weights)

0.7776598703606105

Let us now use the Quantiacs **backtester** for avoiding **forward looking**.

The backtester performs some transformations: it trains the model on one slice of data (using only data from the past) and predicts the weights for the following slice on a rolling basis:

In [16]:
def train_model(data):
    """Create and train the model working on an asset-by-asset basis."""

    asset_name_all = data.coords["asset"].values
    features_all   = get_features(data)
    target_all     = get_target_classes(data)

    models = dict()

    for asset_name in asset_name_all:

        # drop missing values:
        target_cur   = target_all.sel(asset=asset_name).dropna("time", "any")
        features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

        target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

        if len(features_cur.time) < 10:
                continue

        model = get_model()

        try:
            model.fit(feature_for_learn_df.values, target_for_learn_df)
            models[asset_name] = model

        except:
            logging.exception("model training failed")

    return models

# from sklearn.preprocessing import StandardScaler

# def train_model(stock_data):
#     """Create and train the model working on an asset-by-asset basis with data preprocessing."""

#     asset_name_all = stock_data.coords["asset"].values
#     features_all   = get_features(stock_data)
#     target_all     = get_target_classes(stock_data)

#     models = dict()

#     for asset_name in asset_name_all:
#         try:
#             # Imputación de valores faltantes en características
#             target_cur = target_all.sel(asset=asset_name).dropna("time", "any")
#             features_cur = features_all.sel(asset=asset_name).ffill("time").bfill("time").fillna(0)

#             # Alinear características y objetivo
#             target_for_learn_df, feature_for_learn_df = xr.align(target_cur, features_cur, join="inner")

#             # Verificación de datos mínimos para entrenamiento
#             if len(feature_for_learn_df.time) < 30:
#                 logging.warning(f"Activo {asset_name} tiene datos insuficientes para entrenar.")
#                 continue

#             # Escalado de características
#             scaler = StandardScaler()
#             X_scaled = scaler.fit_transform(feature_for_learn_df.to_pandas())

#             # Entrenamiento del modelo
#             model = get_model()
#             model.fit(X_scaled, target_for_learn_df.to_pandas().values.ravel())

#             # Guardar el modelo junto con el escalador
#             models[asset_name] = {"model": model, "scaler": scaler}

#         except Exception as e:
#             logging.exception(f"Fallo en entrenamiento del modelo para {asset_name}: {e}")

#     logging.info(f"Modelos entrenados para {len(models)} activos exitosamente.")
#     return models


In [17]:
def predict_weights(models, data):
    """The model predicts if the price is going up or down.
       The prediction is performed for several days in order to speed up the evaluation."""

    asset_name_all = data.coords["asset"].values
    weights = xr.zeros_like(data.sel(field="close"))

    for asset_name in asset_name_all:
        if asset_name in models:
            model = models[asset_name]
            features_all = get_features(data)
            features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

            if len(features_cur.time) < 1:
                continue

            try:
                weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = model.predict(features_cur.values)

            except KeyboardInterrupt as e:
                raise e

            except:
                logging.exception("model prediction failed")

    return weights

# def predict_weights(models, stock_data):
#     """The model predicts if the price is going up or down.
#        The prediction is performed for several days in order to speed up the evaluation."""

#     asset_name_all = stock_data.coords["asset"].values
#     weights = xr.zeros_like(stock_data.sel(field="close"))

#     features_all = get_features(stock_data)

#     for asset_name in asset_name_all:
#         if asset_name in models:
#             model_info = models[asset_name]
#             model = model_info["model"]
#             scaler = model_info["scaler"]

#             features_cur = features_all.sel(asset=asset_name).dropna("time", "any")

#             if len(features_cur.time) < 1:
#                 logging.warning(f"No hay suficientes datos para predecir en el activo {asset_name}")
#                 continue

#             try:
#                 # Aplicar el mismo escalado que se usó en el entrenamiento
#                 features_scaled = scaler.transform(features_cur.to_pandas())

#                 # Realizar la predicción
#                 predicted_values = model.predict(features_scaled)

#                 # Asignar los pesos a las fechas correspondientes
#                 weights.loc[dict(asset=asset_name, time=features_cur.time.values)] = predicted_values

#             except KeyboardInterrupt as e:
#                 raise e  # Permitir interrupciones manuales

#             except Exception as e:
#                 logging.exception(f"Fallo en la predicción para {asset_name}: {e}")

#     return weights


In [18]:
print("Campos disponibles en los datos:", stock_data.field.values)


Campos disponibles en los datos: ['open' 'low' 'high' 'close' 'vol' 'divs' 'split_cumprod' 'is_liquid'
 'atr' 'ma_50' 'ma_200' 'volatility']


In [19]:
# # Calculate weights using the backtester:
# weights = qnbt.backtest_ml(
#     train                         = train_model,
#     predict                       = predict_weights,
#     train_period                  =  2 *365,  # the data length for training in calendar days
#     retrain_interval              = 10 *365,  # how often we have to retrain models (calendar days)
#     retrain_interval_after_submit = 1,        # how often retrain models after submission during evaluation (calendar days)
#     predict_each_day              = False,    # Is it necessary to call prediction for every day during backtesting?
#                                               # Set it to True if you suspect that get_features is looking forward.
#     competition_type              = "stocks_nasdaq100",  # competition type
#     lookback_period               = 365,                 # how many calendar days are needed by the predict function to generate the output
#     start_date                    = "2005-01-01",        # backtest start date
#     analyze                       = True,
#     build_plots                   = True  # do you need the chart?
# )



weights = qnbt.backtest_ml(
    train                         = train_model,
    predict                       = predict_weights,
    train_period                  =  5 * 365,  # Aumentado a 5 años para entrenar con más datos recientes
    retrain_interval              = 3 * 365,  # Retraining cada 2 años para adaptación a nuevas condiciones de mercado
    retrain_interval_after_submit = 2,        
    predict_each_day              = False,  # Activado para reflejar cambios diarios en el mercado
    competition_type              = "stocks_nasdaq100",
    lookback_period               = 365,                 
    start_date                    = "2006-01-01", # Reducido para entrenar con datos más recientes, y disminuir el tiempo de ejecución
    analyze                       = True,
    build_plots                   = True  
)

# weights = qnbt.backtest_ml(
#     train                         = train_model,
#     predict                       = predict_weights,
#     train_period                  = 5 * 365,  # Incrementado a 5 años para un entrenamiento más robusto
#     retrain_interval              = 2 * 365,  # Reentrenar cada 2 años
#     retrain_interval_after_submit = 30,       # Reentrenar mensualmente después de la entrega
#     predict_each_day              = True,     # Predecir cada día para evitar problemas de forward-looking
#     competition_type              = "stocks_nasdaq100",  # Cambiado para cumplir con Quantiacs
#     lookback_period               = 180,      # Reducido a 180 días para pruebas más rápidas
#     start_date                    = "2005-01-01",  # Fecha de inicio del backtest
#     analyze                       = True,
#     build_plots                   = True  # Se generan gráficos de rendimiento
# )



Run the last iteration...
fetched chunk 1/2 0s
fetched chunk 2/2 0s
Data loaded 0s
fetched chunk 1/1 0s
Data loaded 0s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
WARNING! Strategy trades non-liquid assets.
Fix liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.


NOTICE: The environment variable OUTPUT_PATH was not specified. The default value is 'fractions.nc.gz'


Write output: fractions.nc.gz


NOTICE: The environment variable OUT_STATE_PATH was not specified. The default value is 'state.out.pickle.gz'


State saved.
---
Run First Iteration...
fetched chunk 1/2 0s
fetched chunk 2/2 0s
Data loaded 0s
---
Run all iterations...
Load data...
fetched chunk 1/9 0s
fetched chunk 2/9 0s
fetched chunk 3/9 0s
fetched chunk 4/9 0s
fetched chunk 5/9 0s
fetched chunk 6/9 0s
fetched chunk 7/9 0s
fetched chunk 8/9 0s
fetched chunk 9/9 0s
Data loaded 1s
fetched chunk 1/7 0s
fetched chunk 2/7 0s
fetched chunk 3/7 0s
fetched chunk 4/7 0s
fetched chunk 5/7 0s
fetched chunk 6/7 0s
fetched chunk 7/7 0s
Data loaded 0s
Backtest...


...

Write output: fractions.nc.gz


NOTICE: The environment variable OUT_STATE_PATH was not specified. The default value is 'state.out.pickle.gz'


State saved.
---
Analyze results...
Check...
Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...
Period: 2006-01-01 - 2025-01-16
Sharpe Ratio = 0.7970208069130423
Ok.
---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Select the asset (or leave blank to display the overall stats):


interactive(children=(Combobox(value='', description='asset', options=('', 'NAS:AAL', 'NAS:AAPL', 'NAS:ABNB', …

100% (4791 of 4791) |####################| Elapsed Time: 0:12:35 Time:  0:12:35


The Sharpe ratio is obviously smaller as the training process is not looking forward (as it happens by processing data on a global basis), but performed on a rolling basis.